### Questions we have asked
1. What is the current inventory of a particular store?
2. What are the top-selling products at a particular store?
3. Which store has the highest total sales revenue?
4. What are the 5 stores with the most sales so far this month?
5. How many customers are currently enrolled in the frequent-shopper program?
6. What is the average order value for online orders compared to in-store purchases?
7. Which products have the highest profit margin across all stores?
8. How does the sales performance of a particular product compare between different store locations?
9. Which store locations have the highest percentage of repeat customers?
10. What are the most popular product combinations purchased together by customers?

### Import data

In [ ]:
import pandas as pd

df = pd.read_csv('../webscrape/data/products.csv')

print(df.to_string()) 